Установка библиотек

In [ ]:
!pip install transformers[sentencepiece]
!pip install torch
!pip install transformers[torch]

In [ ]:
import transformers

In [ ]:
!pip install -U transformers datasets evaluate seqeval

In [ ]:
import pandas as pd
import evaluate

from datasets import Dataset
from datasets import ClassLabel

from transformers import AutoModelForTokenClassification, Trainer, AutoTokenizer, DataCollatorForTokenClassification

# Huggin Face

Что такое NLP?

NLP - область лингвистики и машинного обучения, которая изучает все, что связано с естественными языками. Главная цель NLP не просто понимать отдельные слова, но и иметь возможность понимать контекст, в котором эти слова находятся.

Подготовка дата сета. Пример

In [ ]:
def create_custom_dataset(data, labels):
    """
    Create a custom dataset with the given data and labels.

    Args:
    - data: A list of sentences.
    - labels: A list of corresponding labels for each sentence.

    Returns:
    A `datasets.Dataset` object with the given data and labels.
    """
    return datasets.Dataset.from_dict({"text": data, "label": labels})


In [ ]:
import datasets

data = [
    "This is a book.",
    "You are rude.",
    "I am ironic.",
]

labels = ["book", "rude", "ironic"]

custom_dataset = create_custom_dataset(data, labels)

Zero-Shot Classification

In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")
classifier(
    "Осуществлять деятельность, работать, действовать в какой-нибудь области. Подвизается на своём поприще уже четверть столетия. Партию нельзя рассматривать, как нечто оторванное от окружающих людей. Она живёт и подвизается внутри окружающей ее среды.",
    candidate_labels=["книжный", "ироничный", "грубый"],
)

{'sequence': 'Осуществлять деятельность, работать, действовать в какой-нибудь области. Подвизается на своём поприще уже четверть столетия. Партию нельзя рассматривать, как нечто оторванное от окружающих людей. Она живёт и подвизается внутри окружающей ее среды.',
 'labels': ['ироничный', 'грубый', 'книжный'],
 'scores': [0.452871710062027, 0.3673214018344879, 0.1798068732023239]}

Пайплайн question-answering позволяет сгенерировать ответ на вопрос по данному контексту:

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

Модели-кодировщики: полезны для задач, требующих понимания входных данных, таких как классификация предложений и распознавание именованных сущностей.

Для нашего примера нам понадобится модель с головой классификации последовательности (чтобы иметь возможность классифицировать предложения как положительные или отрицательные). Поэтому мы будем использовать не класс AutoModel, а AutoModelForSequenceClassification:

Загрузить уже обученную модель Transformer очень просто - мы можем сделать это с помощью метода from_pretrained():





In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)

# 3 предобучение модели

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'idx': Value(dtype='int32', id=None)}

Раздел 3 Fine tuning с помощью trainer api

# Пример Stackoverflow

In [ ]:
# Define a Classlabel object to use to map string labels to integers.
classmap = ClassLabel(num_classes=4, names=['B-LOC', 'B-PER', 'B-FOOD', 'O'])

train_sentences = [
{'text': 'I live in Madrid', 'labels':['O', 'O', 'O', 'B-LOC']},
{'text': 'Peter lives in Spain', 'labels':['B-PER', 'O', 'O', 'B-LOC']},
{'text': 'He likes pasta', 'labels':['O', 'O', 'B-FOOD']},
]

eval_sentences = [
    {"text": "I like pasta from Madrid , Spain", 'labels': ['O', 'O', 'B-FOOD', 'O', 'B-LOC', 'O', 'B-LOC']}
]

ds_train = Dataset.from_pandas(pd.DataFrame(data=train_sentences))
ds_eval = Dataset.from_pandas(pd.DataFrame(data=eval_sentences))

model = AutoModelForTokenClassification.from_pretrained("distilbert-base-multilingual-cased",
                                                        id2label={i:classmap.int2str(i) for i in range(classmap.num_classes)},
                                                        label2id={c:classmap.str2int(c) for c in classmap.names},
                                                        finetuning_task="ner")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")
data_collator = DataCollatorForTokenClassification(tokenizer)


ds_train = ds_train.map(lambda x: tokenizer(x["text"], truncation=True))
ds_eval = ds_eval.map(lambda x: tokenizer(x["text"], truncation=True))

ds_train = ds_train.map(lambda y: {"labels": classmap.str2int(y["labels"])})
ds_eval = ds_eval.map(lambda y: {"labels": classmap.str2int(y["labels"])})


metric = evaluate.load("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Initialize our Trainer
trainer = Trainer(
    model=model,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


trainer.train()

# Определение словарных помет на основе примера

In [ ]:
!pip install -U transformers datasets evaluate seqeval

In [ ]:
!pip install accelerate


In [ ]:
!pip install torch
!pip install transformers[torch]
!pip install accelerate -U

import pandas as pd
import evaluate
import accelerate
from datasets import Dataset
from datasets import ClassLabel

from transformers import AutoModelForTokenClassification, Trainer, AutoTokenizer, DataCollatorForTokenClassification

# Define a Classlabel object to use to map string labels to integers.
classmap = ClassLabel(num_classes=4, names=['Книжный', 'Ироничный', 'Религиозный', 'Грубый'])

train_sentences = [
{'text': 'Осуществлять деятельность, работать, действовать в какой-нибудь области', 'labels':['Книжный','Ироничный']},
{'text': 'Совершать подвиг в чём-либо, часто о ежедневном борении', 'labels':['Религиозный']},
{'text': 'То же, что замолчать; перестать говорить, кричать, плакать замолкнуть', 'labels':['Грубый']},
]

eval_sentences = [
    {"text": "Сильно изругать", 'labels': ['Грубый']}
]

ds_train = Dataset.from_pandas(pd.DataFrame(data=train_sentences))
ds_eval = Dataset.from_pandas(pd.DataFrame(data=eval_sentences))

model = AutoModelForTokenClassification.from_pretrained("distilbert-base-multilingual-cased",
                                                        id2label={i:classmap.int2str(i) for i in range(classmap.num_classes)},
                                                        label2id={c:classmap.str2int(c) for c in classmap.names},
                                                        finetuning_task="ner")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")
data_collator = DataCollatorForTokenClassification(tokenizer)


ds_train = ds_train.map(lambda x: tokenizer(x["text"], truncation=True))
ds_eval = ds_eval.map(lambda x: tokenizer(x["text"], truncation=True))

ds_train = ds_train.map(lambda y: {"labels": classmap.str2int(y["labels"])})
ds_eval = ds_eval.map(lambda y: {"labels": classmap.str2int(y["labels"])})


metric = evaluate.load("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


trainer = Trainer(
    model=model,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


trainer.train()

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

# Модели Hugging Face для русского языка

distilbert-base-multilingual-cased

[Модели для русского языка Token classification](https://huggingface.co/models?pipeline_tag=token-classification&language=ru&sort=downloads)

[Модели для русского языка Zero-Shot-Classificatiom](https://huggingface.co/models?pipeline_tag=zero-shot-classification&language=ru&sort=downloads)

# Тесты

Тест1. Присвоение предложению несколько классов

In [ ]:
!pip install transformers datasets evaluate

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset, DatasetDict
import numpy as np

Загрузка набора данных и токенизация

In [ ]:
!wget https://github.com/componavt/neural_synset/raw/master/data/label_meaning.csv

In [ ]:
dataset = load_dataset("csv", data_files="label_meaning.csv", sep="|")

class MyTokenizer(AutoTokenizer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

tokenized_datasets = dataset.map(tokenize, batched=True)

Подготовка набора данных для обучения и проверки

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.where(predictions > 0.5, 1, 0)
    return {'accuracy': np.mean(predictions == labels)}

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    evaluation_strategy='epoch',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
)

Измените модель DistilBERT для использования нескольких меток. ???

In [ ]:
class DistilBertForMultiLabelSequenceClassification(AutoModelForSequenceClassification):
    def __init__(self, num_labels, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_labels = num_labels
        self.classifier = torch.nn.Linear(self.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None, **kwargs):
        outputs = super().forward(input_ids, attention_mask=attention_mask, **kwargs)
        logits = self.classifier(outputs.last_hidden_state)

        if labels is not None:
            loss_fct = torch.nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.float().view(-1, self.num_labels))
            outputs = (loss,) + outputs

        return outputs

model = DistilBertForMultiLabelSequenceClassification(num_labels=4)

In [ ]:
trainer.train()

Тест2

Для использования собственного датасета с собственными классами для классификации с помощью Hugging Face Transformers, вам нужно выполнить следующие шаги:

1. Подготовьте свой датасет в формате, который может быть принят библиотекой Transformers (например, CSV, JSON).

2. Загрузите предварительно обученную модель, которую вы хотите дообучить, например, с помощью pipeline из библиотеки Transformers.

3. Дообучите модель на своем датасете с помощью метода train или fine-tune.

4. Протестируйте работу вашей модели на тестовом наборе данных.

In [ ]:
!pip install transformers

In [ ]:
!wget https://github.com/componavt/neural_synset/raw/master/data/label_meaning.csv

In [ ]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
import torch

# Подготовка датасета и модели
train_dataset = ...
test_dataset = ...
# Выбрать модель.
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

# Дообучение модели
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

# Тестирование модели
predictions = trainer.predict(test_dataset)

Тест3 Попытка работы с текущим датасетом ?????

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
# Загрузка данных из CSV файла
url = 'https://github.com/componavt/neural_synset/raw/master/data/label_meaning.csv'
df = pd.read_csv(url, sep='|')

In [ ]:
# Объединение толкования и пометов в один текстовый столбец
df['text'] = df['meaning'] + ' ' + df['книжн.'].astype(str) + ' ' + df['ирон.'].astype(str) + ' ' + df['религ.'].astype(str) + ' ' + df['груб.'].astype(str)

In [ ]:
# Создание TF-IDF векторизатора
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])


In [ ]:
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, df[['книжн.', 'ирон.', 'религ.', 'груб.']], test_size=0.2, random_state=42)

In [ ]:
# Обучение модели
model = OneVsRestClassifier(LogisticRegression())
model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 0 is present in all training examples.
  warnings.warn(


OneVsRestClassifier(estimator=LogisticRegression())

In [ ]:
new_word_meaning = ["дитятя|дитя, ребёнок, чадо"]
new_word_text = [meaning + ' 0 0 0 0' for meaning in new_word_meaning]
new_word_vector = vectorizer.transform(new_word_text)
predictions = model.predict(new_word_vector)

In [ ]:
print(predictions)

In [ ]:
print(df)

Чтобы протестировать модель на тестовом наборе данных и оценить ее производительность, вам следует выполнить следующие шаги:

1. Преобразовать тестовые данные в векторы с помощью того же объекта TfidfVectorizer, который был использован для обучающих данных.
2. Использовать метод predict модели для предсказания меток на тестовом наборе.
3. Сравнить предсказанные метки с фактическими метками из тестового набора и оценить производительность модели.

Вот как это можно сделать:

In [ ]:
# Предсказание меток на тестовом наборе
predictions_test = model.predict(X_test)

# Оценка производительности модели
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, predictions_test)
report = classification_report(y_test, predictions_test)

print("Accuracy:", accuracy)
print("Classification Report:")
print(report)

Accuracy: 0.0
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0

   micro avg       0.00      0.00      0.00         4
   macro avg       0.00      0.00      0.00         4
weighted avg       0.00      0.00      0.00         4
 samples avg       0.00      0.00      0.00         4



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: